# TITLE


Ryan Timbrook (RTIMBROO)  
DATE:
Topic:  


## 1. Objective
_____________________________________________________________________________________________
place object and info here  





## Findings / Recommendations
place findings and recommendations here  





___________________________________________________________________________________________
## FOR RUNNING IN GOOGLE COLAB ONLY ##

In [ ]:
# toggle for working with colab
isColab = False

In [ ]:
#*ONLY RUN WHEN WORKING ON COLAB*
#===================================================
# mount google drive for working in colab

#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)

# working within colab, set base working directory
#base_dir = "./gdrive/My Drive/IST707_PRJ_Realestate/buy_rent_sell/"

# validate directory mapping
#ls f'{base_dir}'

# upload custome python files
#from google.colab import files
#uploaded_files = files.upload()

# print files uploaded
#for f in uploaded_files.keys():
#  print(f'file name: {f}')

#isColab = True

______________________________________________________________________________________________
### Coding Environment Setup
Import packages

In [ ]:
# import packages for analysis and modeling
import pandas as pd #data frame operations
import numpy as np #arrays and math functions
from scipy.stats import uniform #for training and test splits
from scipy.stats import gaussian_kde as kde # for resampling dataset
from scipy import stats #
import statsmodels.formula.api as smf #R-like model specification
import matplotlib.pyplot as plt #2D plotting
%matplotlib inline
import seaborn as sns #
import requests
import os
import pickle


In [ ]:
# packages for twitter
import tweepy as tw
from tweepy import OAuthHandler
import json
from tweepy import Stream
from tweepy.streaming import StreamListener

# Twython packages for twitter
from twython import Twython

# packages for NLTK
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud, STOPWORDS
import re

import sys
from os import path

In [ ]:
# custome python packages
import rtimbroo_utils as br             # custome python helper functions

In [ ]:
# set global properties
notebook_file_name = 'hw1_twitter_sentiment'
report_file_name = 'HW_1_Timbrook_Ryan'
app_name = 'AI_Public_Sentiment'
log_level = 10 # 10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL

# setup working directory structure
# set global properties
if not isColab:
    dataDir = './data'
    outputDir = './output'
    configDir = './config'
    logOutDir = './logs'
    imageDir = './images'
    modelDir = './models'
    corpusDir = './corpus'
else:
    # working within colab
    dataDir = f'{base_dir}data'
    outputDir = f'{base_dir}output'
    configDir = f'{base_dir}config'
    logOutDir = f'{base_dir}logs'
    imageDir = f'{base_dir}images'
    modelDir = f'{base_dir}models'
    corpusDir = f'{base_dir}corpus'

In [ ]:
# create base output directories if they don't exist
if not os.path.exists(outputDir): os.mkdir(outputDir)
if not os.path.exists(logOutDir): os.mkdir(logOutDir)
if not os.path.exists(imageDir): os.mkdir(imageDir)
if not os.path.exists(modelDir): os.mkdir(modelDir)
if not os.path.exists(dataDir): os.mkdir(dataDir)
if not os.path.exists(configDir): os.mkdir(configDir)
if not os.path.exists(corpusDir): os.mkdir(corpusDir)

In [ ]:
# get a logger for troubleshooting / data exploration
logger = br.getFileLogger(logOutDir+'/',app_name,level=log_level)
np.random.seed(42) # NumPy

In [ ]:
# twitter API secret/key
#import json

# Enter your keys/secrets as strings in the following fields
#credentials = {}
#credentials['CONSUMER_KEY'] = ''
#credentials['CONSUMER_SECRET'] = ''
#credentials['ACCESS_TOKEN'] = '-'
#credentials['ACCESS_SECRET'] = ''

# Save the credentials object to file
#with open(f'{configDir}/twitter_credentials.json', "w+") as file:
    #json.dump(credentials, file)

## 2. OBTAIN the data   
________________________________________________________________________________________________
Import external datasets for evaluation

In [ ]:
# load twitter credentials
with open(f'{configDir}/twitter_credentials.json', 'r') as f:
    tw_cred = json.load(f)

# instantiate object
py_tweets = Twython(tw_cred['CONSUMER_KEY'],tw_cred['CONSUMER_SECRET'],tw_cred['ACCESS_TOKEN'],tw_cred['ACCESS_SECRET'])

# setup base twitter search query
search_terms = 'Artificial+Intelligence OR machine+learning'
search_start_date = '2019-10-9'
# add filters to search criteria
filtered_search_terms = search_terms + " -filter:retweets"
# number of tweets to return
num_tweets = 10000

# query
base_tw_query = {
    'q':filtered_search_terms,
    'since':search_start_date,
    'count':num_tweets,
    'lang':'en',
}

# output file names
tweet_filename=f'{dataDir}/{search_start_date}_ai_tweet_text.txt'
raw_filename=f'{dataDir}/{search_start_date}_ai_tweet_raw.txt'

In [ ]:
tw_cred

In [ ]:
import time
# search tweets
tweets_dict = {'user':[],'date':[],'text':[],'favorite_count':[]}

tweet_iterations=10
with open(raw_filename,'w+') as f:
    for i in range(1,tweet_iterations):
        try:
            for status in py_tweets.search(**base_tw_query)['statuses']:
                tweets_dict['user'].append(status['user']['screen_name'])
                tweets_dict['date'].append(status['created_at'])
                tweets_dict['text'].append(status['text'])
                tweets_dict['favorite_count'].append(status['favorite_count'])
            time.sleep(10)
        except BaseException as be:
            print(f'Caught Exception:\n {be}')
            
    tweets_df = pd.DataFrame(tweets_dict)
    tweets_df.sort_values(by='date', inplace=True, ascending=False)
    #tweets_j = json.load(tweets_dict)
    json.dump(tweets_dict,f)
    print(tweets_df.head())

In [ ]:
tweets_df.shape

In [ ]:
# load twitter raw 
with open(f'{dataDir}/{search_start_date}_ai_tweet_raw.txt', 'r') as f:
    j_tweets = json.load(f)

In [ ]:
tweets_j_df = pd.DataFrame(j_tweets)
tweets_j_df.head()

In [ ]:
d = tweets_j_df.sort_values(by='date', inplace=True, ascending=False)
d

In [ ]:
len(tweets_j_df.text.unique())

In [ ]:
dev save_to_file(self,tweet):
    with open(f'{dataDir}/saved_tweets.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(list(tweet.values()))

In [ ]:
# save tweets off to a file
tweet_file_name = 'ai_tweets.txt'
raw_tweet_file_name = 'raw_ai_tweets.txt'
failed_tweets_text = []
failed_count = 0

try:
    with open(dataDir+'/'+tweet_file_name, 'w+') as f:
        for t in tweets:
            try:
                f.write(t.text+'\n')
            except:
                failed_count = failed_count + 1
                failed_tweets_text.append(t.text)
                f.write('\n')
                print(f'Failed to write tweet:\n{t.text}')
            
except BaseException as be:
    print(f'Caught BaseException:\n {be}')
    pass

print(f'Failed to write {failed_count}')


In [ ]:
%%time
# read tweets text file
line_count=0
hash_count=0
word_count=0
tweet_index=0
bag_of_words=[]
bag_of_hashes=[]
bag_of_links=[]
tweets_cleaned={}
tweet_hashs={}
tweet_links={}

# setup some regular expressions
regex1=re.compile('^#.+')
regex2=re.compile('[^\W\d]') #no numbers
regex3=re.compile('^http*')
regex4=re.compile('.+\..+')

with open(f'{dataDir}/{tweet_file_name}') as f:
    for i, line in enumerate(f):
        if line == '\n': 
            tweet_index+=1
            
        #print(line)
        #line = line.replace('\n','')
        line = line.strip()
        line_count+=1
        
        tweetSplitter = TweetTokenizer(strip_handles=True, reduce_len=True)
        tweetTokens = tweetSplitter.tokenize(line)
        
        for token in tweetTokens:
            if(len(token)>2):
                logger.debug(f'Token: {token}')
                
                if((re.match(regex1,token))):
                    logger.debug(f'Token: {token} | matched regex1 {regex1} - will be added to bag of hashes list')
                    n_token=token[1:]
                    bag_of_hashes.append(n_token) # capture all hashtags
                    hash_count+=1
                elif(re.match(regex2,token)):
                    logger.debug(f'Token: {token} | matched regex2 {regex2}')
                    if(re.match(regex3,token) or re.match(regex4,token)):
                        logger.debug(f'Token: {token} | matched regex3 {regex3} or regex4 {regex4} - will be added to bag of links list')
                        bag_of_links.append(token) # capture all url links
                    else:
                        logger.debug(f'Token: {token} | did not match any of the regex patterns - will be added to bag of words list')
                        bag_of_words.append(token) # capture all words
                        word_count+=1
                else:
                    logger.debug(f'Token: {token} | did not match regex2 {regex2} | contains numbers')
                    pass
            else:
                logger.debug(f'Token: {token} | is less than 2')
                pass
        # capture tweet index
        tweets_cleaned[tweet_index] = bag_of_words
        tweet_hashs[tweet_index] = bag_of_hashes
        tweet_links[tweet_index] = bag_of_links
        
        #--End For Loop over tokens
    #--End For loop over lines


logger.info(f'bag_of_words count: {len(bag_of_words)} | bag_of_hashes count: {len(bag_of_hashes)} | bag_of_links count: {len(bag_of_links)}')

In [ ]:
tweet_links.keys()

In [ ]:
len(bag_of_links)

### 2.1 SCRUB / CLEAN
Clean and perform initial transformations steps of the data

In [ ]:
# perform scrubbing and cleaning techniques


### 2.2 Initial EXPLORE
Explore the datasets

In [ ]:
# perform exploratory data analysis techiques

## 3. MODEL
_________________________________________________________________________________________________
Create models

### 3.1 Model Analysis
perform model creation and validation techniques

### 3.2 Model Exploration

### 3.3 Model Build

### 3.4 Model Evaluation

### 3.5 Model Results

#### Model Validation
Perform model validations

### iNterpret
Interpret the model results, make knowledge based recommendations

In [ ]:
# perform interpretation steps

# Report Generation
Execute the below cell to create an HTML report

In [ ]:
# for report auto generation
# testing markup report generation
from nbconvert import HTMLExporter
import codecs
import os
import datetime

stamp = datetime.date.today().strftime("%m-%d-%Y")
exporter = HTMLExporter(template_file='report.tpl')
output, resources = exporter.from_filename(notebook_file_name)
new_fnw = report_file_name
codecs.open(new_fnw, 'w',encoding='utf-8').write(output)